In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
train_df = pd.read_csv('./data/classify-leaves/train.csv')
train_df.describe()

,image,label
count,18353,18353
unique,18353,176
top,images/0.jpg,maclura_pomifera
freq,1,353


In [3]:
leaves_labels = sorted(list(set(train_df['label'])))
num_classes = len(leaves_labels)
print(num_classes)
leaves_labels[:10]

176


['abies_concolor',
 'abies_nordmanniana',
 'acer_campestre',
 'acer_ginnala',
 'acer_griseum',
 'acer_negundo',
 'acer_palmatum',
 'acer_pensylvanicum',
 'acer_platanoides',
 'acer_pseudoplatanus']

In [4]:
class_to_num = dict(zip(leaves_labels, range(num_classes)))
class_to_num

{'abies_concolor': 0,
 'abies_nordmanniana': 1,
 'acer_campestre': 2,
 'acer_ginnala': 3,
 'acer_griseum': 4,
 'acer_negundo': 5,
 'acer_palmatum': 6,
 'acer_pensylvanicum': 7,
 'acer_platanoides': 8,
 'acer_pseudoplatanus': 9,
 'acer_rubrum': 10,
 'acer_saccharinum': 11,
 'acer_saccharum': 12,
 'aesculus_flava': 13,
 'aesculus_glabra': 14,
 'aesculus_hippocastamon': 15,
 'aesculus_pavi': 16,
 'ailanthus_altissima': 17,
 'albizia_julibrissin': 18,
 'amelanchier_arborea': 19,
 'amelanchier_canadensis': 20,
 'amelanchier_laevis': 21,
 'asimina_triloba': 22,
 'betula_alleghaniensis': 23,
 'betula_jacqemontii': 24,
 'betula_lenta': 25,
 'betula_nigra': 26,
 'betula_populifolia': 27,
 'broussonettia_papyrifera': 28,
 'carpinus_betulus': 29,
 'carpinus_caroliniana': 30,
 'carya_cordiformis': 31,
 'carya_glabra': 32,
 'carya_ovata': 33,
 'carya_tomentosa': 34,
 'castanea_dentata': 35,
 'catalpa_bignonioides': 36,
 'catalpa_speciosa': 37,
 'cedrus_atlantica': 38,
 'cedrus_deodara': 39,
 'cedru

In [5]:
num_to_class = {v: k for k, v in class_to_num.items()}
num_to_class

{0: 'abies_concolor',
 1: 'abies_nordmanniana',
 2: 'acer_campestre',
 3: 'acer_ginnala',
 4: 'acer_griseum',
 5: 'acer_negundo',
 6: 'acer_palmatum',
 7: 'acer_pensylvanicum',
 8: 'acer_platanoides',
 9: 'acer_pseudoplatanus',
 10: 'acer_rubrum',
 11: 'acer_saccharinum',
 12: 'acer_saccharum',
 13: 'aesculus_flava',
 14: 'aesculus_glabra',
 15: 'aesculus_hippocastamon',
 16: 'aesculus_pavi',
 17: 'ailanthus_altissima',
 18: 'albizia_julibrissin',
 19: 'amelanchier_arborea',
 20: 'amelanchier_canadensis',
 21: 'amelanchier_laevis',
 22: 'asimina_triloba',
 23: 'betula_alleghaniensis',
 24: 'betula_jacqemontii',
 25: 'betula_lenta',
 26: 'betula_nigra',
 27: 'betula_populifolia',
 28: 'broussonettia_papyrifera',
 29: 'carpinus_betulus',
 30: 'carpinus_caroliniana',
 31: 'carya_cordiformis',
 32: 'carya_glabra',
 33: 'carya_ovata',
 34: 'carya_tomentosa',
 35: 'castanea_dentata',
 36: 'catalpa_bignonioides',
 37: 'catalpa_speciosa',
 38: 'cedrus_atlantica',
 39: 'cedrus_deodara',
 40: 'c

In [6]:
class LeavesData(Dataset):
    def __init__(self, csv_path, file_path, mode='train', valid_ratio=0.2, resize_height=256, resize_width=256):
        self.resize_height = resize_height
        self.resize_width = resize_width
        
        self.file_path = file_path
        self.mode = mode
        
        self.data_info = pd.read_csv(csv_path, header=None)
        self.data_len = len(self.data_info.index) - 1
        self.train_len = int(self.data_len * (1 - valid_ratio))
        
        if mode == 'train':
            self.train_image = np.asarray(self.data_info.iloc[1:self.train_len, 0])
            self.train_label = np.asarray(self.data_info.iloc[1:self.train_len, 1])
            self.image_arr = self.train_image
            self.label_arr = self.train_label
        elif mode == 'valid':
            self.valid_image = np.asarray(self.data_info.iloc[self.train_len:, 0])
            self.valid_label = np.asarray(self.data_info.iloc[self.train_len:, 1])
            self.image_arr = self.valid_image
            self.label_arr = self.valid_label
        elif mode == 'test':
            self.test_image = np.asarray(self.data_info.iloc[1:, 0])
            self.image_arr = self.test_image
        
        self.real_len = len(self.image_arr)
        
        print('Finished reading the {} set of Leaves Dataset ({} samples found.)'.format(mode, self.real_len))
    
    def __getitem__(self, idx):
        single_image_name = self.image_arr[idx]
        image = Image.open(self.file_path + single_image_name)
        
        if self.mode == 'train':
            transform = transforms.Compose([
                # transforms.Resize((224, 224)),
                transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
            
        image = transform(image)
        
        if self.mode == 'test':
            return image
        else:
            label = self.label_arr[idx]
            number_label = class_to_num[label]
            return image, number_label
    
    def __len__(self):
        return self.real_len

In [7]:
train_path = './data/classify-leaves/train.csv'
test_path = './data/classify-leaves/test.csv'
image_path = './data/classify-leaves/'

train_dataset = LeavesData(train_path, image_path, mode='train')
valid_dataset = LeavesData(train_path, image_path, mode='valid')
test_dataset = LeavesData(test_path, image_path, mode='test')
print(train_dataset)
print(valid_dataset)
print(test_dataset)

Finished reading the train set of Leaves Dataset (14681 samples found.)
Finished reading the valid set of Leaves Dataset (3672 samples found.)
Finished reading the test set of Leaves Dataset (8800 samples found.)


In [8]:
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=0)
val_loader = DataLoader(dataset=valid_dataset, batch_size=128, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False, num_workers=0)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False


def network(num_classes, feature_extract = False, use_pretrained=True):

    # model_ft = models.densenet161(pretrained=use_pretrained)
    model_ft = models.resnet34(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    # num_ftrs = model_ft.classifier.in_features
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [11]:
learning_rate = 1e-4
weight_decay = 1e-3
num_epochs = 30
model_path = './trained_model'

In [12]:
model = network(176)
model = model.to(device)
model.device = device
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

best_acc = 0.0
for epoch in range(num_epochs):
    # start training
    model.train()
    train_loss, train_acc = [], []
    for batch in tqdm(train_loader):
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        
        # prediction
        output = model(images)
        loss = criterion(output, labels)
        
        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (output.argmax(dim=-1) == labels).float().mean()
        
        train_loss.append(loss.item())
        train_acc.append(acc)
        
    mean_train_loss = sum(train_loss) / len(train_loss)
    mean_train_acc = sum(train_acc) / len(train_acc)
    
    print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {mean_train_loss:.5f}, acc = {mean_train_acc:.5f}")
    
    # start validation
    model.eval()
    valid_acc, valid_loss = [], []
    for batch in tqdm(val_loader):
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
            acc = (output.argmax(dim=-1) == labels).float().mean()
            valid_loss.append(loss.item())
            valid_acc.append(acc)
    
    mean_valid_loss = sum(valid_loss) / len(valid_loss)
    mean_valid_acc = sum(valid_acc) / len(valid_acc)
    
    print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {mean_valid_loss:.5f}, acc = {mean_valid_acc:.5f}")
    
    if mean_valid_acc > best_acc:
        best_acc = mean_valid_acc
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc))

print("The highest validation accuracy is: {:.3f}".format(best_acc.item()))

  0%|                                                                                          | 0/115 [00:00<?, ?it/s]d:\miniconda\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  3%|██▊                                                                                | 1/29 [00:00<00:05,  5.26it/s]

[ Train | 001/050 ] loss = 3.15603, acc = 0.36137


  0%|                                                                                          | 0/115 [00:00<?, ?it/s]

[ Valid | 001/050 ] loss = 1.85109, acc = 0.58567
saving model with acc 0.586


  3%|██▊                                                                                | 1/29 [00:00<00:05,  5.33it/s]

...

[ Valid | 049/050 ] loss = 0.31254, acc = 0.90916


  3%|██▊                                                                                | 1/29 [00:00<00:05,  5.29it/s]

[ Train | 050/050 ] loss = 0.09172, acc = 0.97362


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:05<00:00,  5.27it/s]

[ Valid | 050/050 ] loss = 0.24073, acc = 0.93123
The highest validation accuracy is: 0.9328467845916748


In [13]:
SaveFileName = './submission1.csv'

In [18]:
# testing
model = network(176)
model = model.to(device)
model.load_state_dict(torch.load(model_path))
model.eval()
predictions = []
for batch in tqdm(test_loader):
    images = batch
    with torch.no_grad():
        output = model(images.to(device))
    pred_labels = output.argmax(dim=-1)
    predictions.extend(pred_labels.cpu().numpy().tolist())

preds = []
for i in predictions:
    preds.append(num_to_class[i])

test_data = pd.read_csv(test_path)
test_data['label'] = pd.Series(preds)
submission = pd.concat([test_data['image'], test_data['label']], axis=1)
submission.to_csv(SaveFileName, index=False)
print('Test Done!')

100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [00:13<00:00,  5.24it/s]

Test Done!
